# Chapter 11, trees

The chapter starts explaining how we can represent a binary tree in OCaml, it is a simple but good discussion, at the end we will have a polymorphic type holding a branch (with data) and a leaf (signaling it is a end node) so it contained no data:

In [32]:
type 'a tree =
| Branch of 'a * 'a tree * 'a tree
| Leaf 
;;

type 'a tree = Branch of 'a * 'a tree * 'a tree | Leaf


To count the number of elements of a tree we count the number of branches and skip leaves.

In [15]:
let rec size t =
  match t with
  | Branch (_, l, r) -> 1 + size l + size r
  | Leaf -> 0
;;

val size : 'a tree -> int = <fun>


We can calculate the "depth" of a tree, or the longest branch in the tree:

In [16]:
let rec depth t =
  let max a b =
    if a > b then a else b in
  match t with
  | Branch (_, l, r) -> max (depth l) (depth r)
  | Leaf -> 0
;;

val depth : 'a tree -> int = <fun>


Now guess what, because the author fucking love lists (I think he really thinks it is the best data type ever created) now he wants to convert a tree into a list... wonderful.

In [17]:
let rec to_list t =
  match t with
  | Branch (x, l, r) -> x :: (to_list l) @ (to_list r)
  | Leaf -> []
;;

val to_list : 'a tree -> 'a list = <fun>


I am not even try to make it recursive, the author doesn't even try neither that is because he is drinking too many lists...

In [22]:
let rec tree_map f t =
  match t with
  | Branch (x, l, r) -> Branch (f x, tree_map f l, tree_map f r)
  | _ -> Leaf
;;

val tree_map : ('a -> 'b) -> 'a tree -> 'b tree = <fun>


Now we can design a dictionary using a balanced binary tree, to easy the search, take for example the following tree of type `(int * string)`:

In [26]:
Branch ((3, "three"), Branch ((1, "one"), Leaf, Branch ((2, "two"), Leaf, Leaf)), Branch ((4, "four"), Leaf, Leaf))

- : (int * string) tree =
Branch ((3, "three"),
 Branch ((1, "one"), Leaf, Branch ((2, "two"), Leaf, Leaf)),
 Branch ((4, "four"), Leaf, Leaf))


Basically, if you display the tree (I can't and I don't want to install graphviz) it will look that for every branch there is one with the a greater number to the right and lesser number to the left. Easy now to search for a number!

In [29]:
let rec lookup k t =
  match t with
  | Branch ((k', v), l, r) -> 
    if k' = k then v else
    if k' < k then lookup k r else lookup k l
  | _ -> raise (Not_found)
;;

val lookup : 'a -> ('a * 'b) tree -> 'b = <fun>


We can create a function to insert the value into our new dictionary structure (using a balance tree). The same procedure happens, left is "lesser than" and right is "greater than". Of course we have to move the structure if already have a number there.

In [31]:
let rec insert k v t =
  match t with
  | Branch ((k', v'), l, r) ->
    if k = k' then Branch ((k, v), l, r) else
    if k > k' then Branch ((k', v'), l, insert k v r) else
    Branch ((k', v'), insert k v l, r)
  | Leaf -> Branch ((k, v), Leaf, Leaf)
;;

val insert : 'a -> 'b -> ('a * 'b) tree -> ('a * 'b) tree = <fun>


## Exercises

 * Write a function of type `α → α tree → bool` to determine if a given element is in a tree.

In [33]:
let rec tree_contains x t =
  match t with
  | Branch (x', l, r) ->
    if x = x' then true else
    if x < x' then tree_contains x l else
    tree_contains x r
  | Leaf -> false
;;

val tree_contains : 'a -> 'a tree -> bool = <fun>


 * Write a function which flips a tree left to right such that, if it were drawn on paper, it would appear to be a mirror image.

In [36]:
(* This function is not tail recursive *)
let rec flip_tree t =
  match t with
  | Branch (v, l, r) -> Branch (v, flip_tree r, flip_tree l)
  | Leaf -> Leaf
;;

val flip_tree : 'a tree -> 'a tree = <fun>


 * Write a function to determine if two trees have the same shape, irrespective of the actual values of the elements.

In [41]:
let rec same_shape a b =
  match a, b with
  | Branch (_, la, ra), Branch (_, lb, rb) -> (same_shape la lb) && (same_shape ra rb)
  | Leaf, Leaf -> true
  | _, _ -> false
;;

let a = Branch(1, Branch(2, Leaf, Leaf), Leaf) ;;
let b = Branch(100, Branch(200, Leaf, Leaf), Leaf) ;;
let c = Branch(100, Branch(200, Leaf, Leaf), Branch(400, Leaf, Leaf)) ;;

same_shape a b ;;
same_shape a c ;;

val same_shape : 'a tree -> 'b tree -> bool = <fun>


val a : int tree = Branch (1, Branch (2, Leaf, Leaf), Leaf)


val b : int tree = Branch (100, Branch (200, Leaf, Leaf), Leaf)


val c : int tree =
  Branch (100, Branch (200, Leaf, Leaf), Branch (400, Leaf, Leaf))


- : bool = true


- : bool = false


 * Write a function `tree_of_list` which builds a tree representation of a dictionary from a list representation of a dictionary.

In [44]:
let rec tree_of_list l =
  match l with
  | (k, v) :: tl -> insert k v (tree_of_list tl)
  | _ -> Leaf
;;

tree_of_list [(1,"uno"); (5, "cinco"); (3, "tres"); (2, "dos"); (4, "cuatro")]

val tree_of_list : ('a * 'b) list -> ('a * 'b) tree = <fun>


- : (int * string) tree =
Branch ((4, "cuatro"),
 Branch ((2, "dos"), Branch ((1, "uno"), Leaf, Leaf),
  Branch ((3, "tres"), Leaf, Leaf)),
 Branch ((5, "cinco"), Leaf, Leaf))


 * Write a function to combine two dictionaries represented as trees into one. In the case of clashing keys, prefer the value from the first dictionary.

In [46]:
let union_tree a b =
  tree_of_list ((to_list a) @ (to_list b))
;;

val union_tree : ('a * 'b) tree -> ('a * 'b) tree -> ('a * 'b) tree = <fun>


 * Can you define a type for trees which, instead of branching exactly two ways each time, can branch zero or more ways, possibly different at each branch? Write simple functions like `size`, `total`, and `map` using your new type of tree.

_I probably could but I won't because I got tired and bored and I want to finish this book so I can read a better one._